In [26]:
#!/usr/bin/python

import sys
import pickle
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import test_classifier, dump_classifier_and_data

#additional imports
import numpy as np
from sklearn.feature_selection import SelectKBest
import enron_tools
import enron_evaluate
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn import preprocessing
from sklearn import cross_validation
import enron_evaluate

sep = '##############################################################################################'
sep2 = '++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++'

### Load the dictionary containing the dataset
data_dict = pickle.load(open("final_project_dataset.pkl", "r") )

### create list of functions for use as argument to add_features function
add_feature_function_list = [enron_tools.add_poi_to_ratio,enron_tools.add_poi_from_ratio,enron_tools.add_poi_interaction_ratio]

## add features to data_dict
enron_tools.add_features(add_feature_function_list,data_dict)

### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".

data_label = 'poi'
features_list = enron_tools.get_features(data_dict)


## email address does not help with prediction and causes exeception, remove
features_list.remove('email_address')


##remove the data_label so that it can be re-added as the first feature element
features_list.remove('poi')

##reassemble feaures with the data label as the first element
features_list = [data_label] + features_list

############################################################################################################

### Task 2: Remove outliers

outliers = ['TOTAL','THE TRAVEL AGENCY IN THE PARK']

enron_tools.remove_outliers(data_dict, outliers)


### Store to my_dataset for easy export below.
my_dataset = data_dict

###############################################################################################

### Continue Feature Selection and dimensionality reduction via get_k_best

## get k (k represents number of features) best features
k = 10
k_best_features = enron_tools.get_k_best(data_dict,features_list,k)

print sep

# assemble feature list
my_features_list = [data_label] + list(k_best_features.feature.values)



###################################################################################################


### extract features and labels for gridsearch optimization

# data extraction using k_best features list
data = featureFormat(my_dataset, my_features_list, sort_keys = True)

tru, trn = targetFeatureSplit(data)

## scale extracted features
scaler = preprocessing.MinMaxScaler()
trn = scaler.fit_transform(trn)


# Set up cross validator (will be used for tuning all classifiers)
cv = cross_validation.StratifiedShuffleSplit(tru,
                                            n_iter = 10,
                                             random_state = 42)

## Final Adaboost Classifier

# Adaboost Classifier
from sklearn.ensemble import AdaBoostClassifier
a_clf = AdaBoostClassifier()

# set up estimator and pipeline, using PCA for feature selection
estimators = [('reduce_dim', PCA()),('adaboost',a_clf)]
aclf = Pipeline(estimators)

# set up paramaters dictionary
a_params = dict(reduce_dim__n_components=[perc_var],
              adaboost__n_estimators=[5, 10, 30, 40, 50, 100, 150, 200],
                  adaboost__learning_rate=[0.1, 0.5, 1, 1.5, 2, 2.5],
                   adaboost__algorithm=('SAMME', 'SAMME.R'))

# set up gridsearch
a_grid_search = GridSearchCV(aclf, param_grid = a_params,
                          scoring = 'f1', cv =cv)
# pass data into into the gridsearch via fit
a_grid_search.fit(trn,tru)

best_aclf = a_grid_search.best_estimator_

## Evaluate Tuned Adaboost Classifier


best_a_pipe_scale = Pipeline(steps=[('scale_features',preprocessing.MinMaxScaler()),('pca',PCA(n_components=.95)),('adaboost',best_aclf)])


print 'best_a_clf\n'
test_classifier(best_a_pipe,my_dataset,my_features_list)
print sep

### Dump classifier for use in final algorithm poi_id

pickle.dump(best_a_pipe_scale, open('best_clf_pipe.pkl', "w") )


Feature Selection by k_best_features

10 best features in descending order: ['exercised_stock_options' 'total_stock_value' 'bonus' 'salary'
 'deferred_income' 'poi_ratio' 'long_term_incentive' 'restricted_stock'
 'total_payments' 'shared_receipt_with_poi']

                   feature      score  percent_nan
0  exercised_stock_options  24.815080         29.9
1        total_stock_value  24.182899         13.2
2                    bonus  20.792252         43.8
3                   salary  18.289684         34.7
4          deferred_income  11.458477         66.7
5                poi_ratio  10.019415         40.3
6      long_term_incentive   9.922186         54.9
7         restricted_stock   9.212811         24.3
8           total_payments   8.772778         14.6
9  shared_receipt_with_poi   8.589421         40.3

##############################################################################################
best_a_clf

Pipeline(steps=[('scale_features', MinMaxScaler(copy=True, feature_range=

In [25]:
#!/usr/bin/python

import sys
import pickle
sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import test_classifier, dump_classifier_and_data

#additional imports
import numpy as np
from sklearn.feature_selection import SelectKBest
import enron_tools
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn import preprocessing
from sklearn import cross_validation
from sklearn.ensemble import AdaBoostClassifier

sep = '##############################################################################################'
sep2 = '++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++'

### Load the dictionary containing the dataset
data_dict = pickle.load(open("final_project_dataset.pkl", "r") )

### create list of functions for use as argument to add_features function
add_feature_function_list = [enron_tools.add_poi_to_ratio,enron_tools.add_poi_from_ratio,enron_tools.add_poi_interaction_ratio]

## add features to data_dict
enron_tools.add_features(add_feature_function_list,data_dict)

### Task 1: Select what features you'll use.
### features_list is a list of strings, each of which is a feature name.
### The first feature must be "poi".

data_label = 'poi'
features_list = enron_tools.get_features(data_dict)


## email address does not help with prediction and causes exeception, remove
features_list.remove('email_address')


##remove the data_label so that it can be re-added as the first feature element
features_list.remove('poi')

##reassemble feaures with the data label as the first element
features_list = [data_label] + features_list

############################################################################################################

### Task 2: Remove outliers

outliers = ['TOTAL','THE TRAVEL AGENCY IN THE PARK']

enron_tools.remove_outliers(data_dict, outliers)


### Store to my_dataset for easy export below.
my_dataset = data_dict

###############################################################################################

### Continue Feature Selection and dimensionality reduction via get_k_best

## get k (k represents number of features) best features
k = 10
k_best_features = enron_tools.get_k_best(data_dict,features_list,k)

print sep

# assemble feature list
my_features_list = [data_label] + list(k_best_features.feature.values)



###################################################################################################


### extract features and labels for gridsearch optimization

# data extraction using k_best features list
data = featureFormat(my_dataset, my_features_list, sort_keys = True)

tru, trn = targetFeatureSplit(data)

## scale extracted features
scaler = preprocessing.MinMaxScaler()
trn = scaler.fit_transform(trn)


# Set up cross validator (will be used for tuning all classifiers)
cv = cross_validation.StratifiedShuffleSplit(tru,
                                            n_iter = 10,
                                             random_state = 42)

## Evaluate Final Adaboost Classifier

# load tuned classifier pipeline


best_a_pipe = pickle.load(open('best_clf_pipe.pkl', "r") )




print 'best_a_clf\n'
best_a_pipe
test_classifier(best_a_pipe,my_dataset,my_features_list)
print sep

### Dump your classifier, dataset, and features_list so 
### anyone can run/check your results.

dump_classifier_and_data(best_a_pipe, my_dataset, my_features_list)


Feature Selection by k_best_features

10 best features in descending order: ['exercised_stock_options' 'total_stock_value' 'bonus' 'salary'
 'deferred_income' 'poi_ratio' 'long_term_incentive' 'restricted_stock'
 'total_payments' 'shared_receipt_with_poi']

                   feature      score  percent_nan
0  exercised_stock_options  24.815080         29.9
1        total_stock_value  24.182899         13.2
2                    bonus  20.792252         43.8
3                   salary  18.289684         34.7
4          deferred_income  11.458477         66.7
5                poi_ratio  10.019415         40.3
6      long_term_incentive   9.922186         54.9
7         restricted_stock   9.212811         24.3
8           total_payments   8.772778         14.6
9  shared_receipt_with_poi   8.589421         40.3

##############################################################################################
best_a_clf

Pipeline(steps=[('scale_features', MinMaxScaler(copy=True, feature_range=